# Preprocesamiento de datos

## Librerías

In [1]:
import pandas as pd
import numpy as np
import random
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

## Funciones

In [2]:
def CargarPandasDataset(ruta):
    """
    Carga un dataset de pandas a partir de un archivo csv con espacios vacios representados por un \r
    Recibe:
        ruta_parcial: string con la ruta parcial o absoluta al archivo csv.
    Devuelve:
        pandas dataset
    """
    return pd.read_csv(ruta, na_values=['\r'])

def GuardarDataset(df, name):
    """
    Guarda el dataset de pandas en la ruta indicada
    Recibe:
          df: dataset de pandas
        name: nombre con el que será guardado
    Devuelve:
        nada
    """
    df.to_csv(name, index=False)
    return

def ModificarColumnasValor(df, cols, valorR, valorN):
    """
    Modifica las columnas indicadas del dataframe de pandas donde un valor deba ser reemplazado por un valor nuevo
    Recibe:
            df: dataset de pandas
          cols: columnas a remplazar valores
        valorR: valor a remplazar
        valorN: valor nuevo
    Devuelve:
        pandas dataframe modificado
    """
    df_copy = copy.copy(df)
    df_copy[cols] = df[cols].replace(valorR, valorN)
    return df_copy

def BorrarColumnas(df, cols):
    """
    Devuelve un pandas dataframe sin las columnas indicadas 
    Recibe:
             df: dataset de pandas
           cols: columnas a remplazar valores
    Devuelve:
         pandas dataframe
    """
    df_new = df.drop(labels=cols, axis=1, inplace=False)
    return df_new

def InsertarColumnaNueva(df, nombreCol, numeroCol, funcion):
    """
    Inserta una columna nueva nombreCol, en la posicion numero col, en el dataset de pandas df, en base a la funcion 
    Recibe:
               df: dataset de pandas
        nombreCol: nombre de la columna nueva
        numeroCol: numero de columna donde se insertará
          funcion: funcion de Python
    Devuelve:
        pandas dataset
    """
    df_copy = copy.copy(df)
    df_copy.insert(numeroCol, nombreCol, df.apply(funcion, axis=1))
    return df_copy

def transformColumn(col):
    """
    Encodea la columna de 0 a n-1, siendo n el numero de clases en la columna usando un LabelEncoder de sklearn llamado lb
    Recibe:
        col: columna de pandas
    Devuelve:
        la columna transformada
    """
    return lb.fit_transform(col)

In [3]:
def RegionesDeMexico(row):
    if   row['NOM_ENT'] in ['Baja California\r', 'Baja California Sur\r', 'Sonora\r', 'Sinaloa\r', 'Nayarit\r']:
        val = 0
    elif row['NOM_ENT'] in ['Chihuahua\r', 'Coahuila de Zaragoza\r', 'Durango\r', 'Zacatecas\r', 'San Luis Potosi\r']:
        val = 1
    elif row['NOM_ENT'] in ['Nuevo Leon\r', 'Tamaulipas\r']:
        val = 2
    elif row['NOM_ENT'] in ['Jalisco\r', 'Aguascalientes\r', 'Guanajuato\r', 'Colima\r', 'Michoacan de Ocampo\r']:
        val = 3
    elif row['NOM_ENT'] in ['Queretaro\r', 'Hidalgo\r', 'Estado de Mexico\r',  'Ciudad de Mexico\r','Morelos\r', 'Puebla\r', 'Tlaxcala\r']:
        val = 4
    elif row['NOM_ENT'] in ['Veracruz de Ignacio de la Llave\r', 'Tabasco\r']:
        val = 5
    elif row['NOM_ENT'] in ['Guerrero\r', 'Oaxaca\r', 'Chiapas\r']:
        val = 6
    elif row['NOM_ENT'] in ['Campeche\r', 'Yucatan\r', 'Quintana Roo\r']:
        val = 7
    return val

def Clasificacion(row):
    # va de menor a mayor gravedad
    val = [0,0] #ninguno
    
    #abuso y violencia
    if row['P9_8_1'] == 1.0 or row['P9_8_2'] == 1.0 or row['P9_8_3'] == 1.0 or row['P9_8_4'] == 1.0 or row['P9_8_5'] == 1.0 or row['P9_8_6'] == 1.0:
        val[0] = 1
    
    #atencion no autorizada
    if row['P9_8_7'] == 1.0 or row['P9_8_8'] == 1.0 or row['P9_8_9'] == 1.0 or row['P9_8_10'] == 1.0 or row['P9_8_14'] == 4.0:
        val[1] = 1
    
    if val[0]==1 and val[1]==1: #ambas
        return 3
    elif val[0]==1: #abuso y violencia
        return 1
    elif val[1]==1: #atencion no autorizada
        return 2
    else: 
        return 0 #ninguno

## Seccion I

### Cargar datos

In [4]:
ruta_parcial = "../../conjunto_de_datos_endireh_2016_csv/conjunto_de_datos_tviv_endireh_2016/conjunto_de_datos/conjunto_de_datos_tviv_endireh_2016.csv"

seccionI = CargarPandasDataset(ruta_parcial)

seccionI.iloc[random.choices(range(seccionI.shape[0]), k=5)]

,ID_VIV,UPM,PROG,VIV_SEL,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,COD_RES,P1_1,...,P1_9,P1_10_1,P1_10_2,P1_10_3,P1_10_4,FAC_VIV,DOMINIO,EST_DIS,UPM_DIS,ESTRATO
68949,1903637.03,1903637,49,3,19,Nuevo Leon\r,26,Guadalupe\r,2,2,...,NaN,NaN,NaN,NaN,NaN,374,U\r,3340,10461,3
15397,500678.05,500678,115,5,5,Coahuila de Zaragoza\r,18,Monclova\r,1,3,...,NaN,NaN,NaN,NaN,NaN,206,U\r,780,2467,2
120157,3202335.18,3202335,72,18,32,Zacatecas\r,44,Tabasco\r,1,3,...,NaN,NaN,NaN,NaN,NaN,111,C\r,6160,17894,2
44118,1260405.18,1260405,265,18,12,Guerrero\r,1,Acapulco de Juárez\r,1,2,...,NaN,NaN,NaN,NaN,NaN,262,R\r,2180,7111,1
35007,1001003.04,1001003,75,4,10,Durango\r,5,Durango\r,1,1,...,NaN,NaN,NaN,NaN,NaN,118,U\r,1620,5860,2


### Hacer columnas One Hot Encoding

In [5]:
columnasOHE = [f'P1_4_{i}' for i in range(1,10)]
columnasOHE.append('P1_8')

seccionI = ModificarColumnasValor(df=seccionI, cols=columnasOHE, valorR=2, valorN=0)

### Política de datos faltantes

In [6]:
seccionI = ModificarColumnasValor(df=seccionI, cols=['P1_9'], valorR=np.nan, valorN=1)

columnasNaN = [f'P1_10_{i}' for i in range(1, 5)]

seccionI = ModificarColumnasValor(df=seccionI, cols=columnasNaN, valorR=np.nan, valorN=2)

### Borrar columnas

In [7]:
labels = ['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'COD_RES', 'UPM', 'PROG','FAC_VIV', 'EST_DIS', 'UPM_DIS', 'ESTRATO', 'DOMINIO']

seccionI = BorrarColumnas(df=seccionI, cols=labels)

### Guardar dataset

In [8]:
GuardarDataset(df=seccionI, name='datasets/sec_i.csv')

## Seccion II

### Cargar los datos

In [9]:
ruta_parcial = "../../conjunto_de_datos_endireh_2016_csv/conjunto_de_datos_tsdem_endireh_2016/conjunto_de_datos/conjunto_de_datos_tsdem_endireh_2016.csv"

seccionII = CargarPandasDataset(ruta_parcial)

seccionII.iloc[random.choices(range(seccionII.shape[0]), k=5)]

,ID_VIV,ID_MUJ,UPM,PROG,VIV_SEL,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,HOGAR,...,REN_INF_AD,FN_DIA,FN_MES,FAC_VIV,FAC_MUJ,DOMINIO,ESTRATO,EST_DIS,UPM_DIS,COD_M15
299110,2160073.17,2160073.17.01.01\r,2160073,9,17,21,Puebla\r,45,Chalchicomula de Sesma\r,1,...,NaN,NaN,NaN,437,0,R\r,1,3820,11939,NaN
166979,1260616.03,1260616.03.01.02\r,1260616,105,3,12,Guerrero\r,12,Ayutla de los Libres\r,1,...,NaN,NaN,NaN,214,0,R\r,1,2180,7124,1.0
404709,2901063.03,2901063.03.01.02\r,2901063,71,3,29,Tlaxcala\r,10,Chiautempan\r,1,...,NaN,NaN,NaN,87,0,U\r,2,5370,16296,NaN
226226,2260201.08,2260201.08.01.02\r,2260201,241,8,22,Queretaro\r,16,San Juan del Río\r,1,...,NaN,NaN,NaN,179,179,R\r,2,4080,12524,1.0
175081,1301843.10,1301843.10.01.02\r,1301843,53,10,13,Hidalgo\r,10,Atitalaquia\r,1,...,2.0,NaN,NaN,203,203,C\r,2,2340,7350,1.0


### Modificar columnas a OHE

In [10]:
columnasOHE = ['P2_8', 'P2_9', 'P2_11', 'P2_12', 'P2_13']

seccionII = ModificarColumnasValor(df=seccionII, cols=columnasOHE, valorR=2, valorN=0)

### Política de datos faltantes

In [11]:
seccionII = ModificarColumnasValor(df=seccionII, cols=['P2_14'], valorR=np.nan, valorN=13)

columnasNaN = ['P2_8', 'P2_12', 'P2_15']

seccionII = ModificarColumnasValor(df=seccionII, cols=columnasNaN, valorR=np.nan, valorN=9)

### 2.2 Dividir por regiones de méxico.

Para no dividir entre 32 entidades federativas, dividiré el país en principales regiones de México.
Según la regionalización económica de la [CONABIO](http://www.conabio.gob.mx/informacion/metadata/gis/recomgw.xml?_xsl=/db/metadata/xsl/fgdc_html.xsl&_indent=no) existen un total de 8 regiones.
   
<img src="conabio.gob.mx/informacion/gis/layouts/recomgw.png"
     alt="Regiones económicas de Mexico"
     style="float: left; margin-right: 10px;" />
     
Cada código representa a una región como se muestra en la siguiente tabla:

|  Código  | Región              |
|:--------:|:-------------------:|
| 0        | Noroeste            |
| 1        | Norte               |
| 2        | Noreste             |
| 3        | Centro Occidente    |
| 4        | Centro Sur          |
| 5        | Golfo de México     |
| 6        | Pacífico Sur        |
| 7        | Península de Yucatán|


In [12]:
seccionII_reg = InsertarColumnaNueva(df=seccionII, nombreCol='REGION', numeroCol=12, funcion=RegionesDeMexico)

### Borrar columnas

In [13]:
labels = ['NOM_ENT', 'NOM_MUN', 'COD_RES', 'COD_RES_E', 'FN_DIA', 'FN_MES', 'NOMBRE', 'HOGAR', 
          'UPM', 'VIV_SEL', 'N_REN', 'REN_MUJ_EL', 'REN_INF_AD', 'FAC_VIV', 'FAC_MUJ',
          'ESTRATO', 'EST_DIS', 'UPM_DIS', 'CODIGO', 'COD_M15', 'PROG', 'P2_16']

seccionII_reg = BorrarColumnas(df=seccionII_reg, cols=labels)

### Pasar columnas categóricas a numéricas

In [14]:
lb = LabelEncoder()

In [15]:
seccionII_reg[['DOMINIO']] = seccionII_reg[['DOMINIO']].apply(transformColumn)
lb.classes_ 

array(['C\r', 'R\r', 'U\r'], dtype=object)

### Guardar dataset

In [16]:
GuardarDataset(df=seccionII_reg, name='datasets/sec_ii.csv')

## Seccion III

### Cargar datos

In [17]:
ruta_parcial = "../../conjunto_de_datos_endireh_2016_csv/conjunto_de_datos_tb_sec_iii_endireh_2016/conjunto_de_datos/conjunto_de_datos_tb_sec_iii_endireh_2016.csv"

seccionIII = CargarPandasDataset(ruta_parcial)

seccionIII.iloc[random.choices(range(seccionIII.shape[0]), k=5)]

,ID_VIV,ID_MUJ,UPM,PROG,VIV_SEL,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,HOGAR,...,P3_5,P3_6,P3_7,P3_8,FAC_VIV,FAC_MUJ,DOMINIO,ESTRATO,EST_DIS,UPM_DIS
100964,3001124.01,3001124.01.01.01\r,3001124,8,1,30,Veracruz de Ignacio de la Llave\r,44,Córdoba\r,1,...,NaN,NaN,NaN,B2\r,518,518,U\r,2,5730,16653
50271,1523706.05,1523706.05.01.02\r,1523706,136,5,15,Estado de Mexico\r,118,Zinacantepec\r,1,...,NaN,NaN,NaN,A1\r,1343,1343,U\r,2,2670,8707
80686,2401537.04,2401537.04.01.01\r,2401537,33,4,24,San Luis Potosi\r,28,San Luis Potosí\r,1,...,NaN,NaN,NaN,B1\r,188,188,U\r,3,4300,13449
32232,1002375.04,1002375.04.01.04\r,1002375,80,4,10,Durango\r,12,Lerdo\r,1,...,NaN,NaN,NaN,A2\r,169,508,U\r,2,1680,6093
75290,2202515.10,2202515.10.01.02\r,2202515,182,10,22,Queretaro\r,5,Colón\r,1,...,NaN,NaN,NaN,A1\r,136,136,C\r,2,4040,12441


### Modificar columnas a OHE

In [18]:
columnasOHE = ['P3_2', 'P3_4', 'P3_7']

seccionIII = ModificarColumnasValor(df=seccionIII, cols=columnasOHE, valorR=2, valorN=0)

### Política de datos faltantes

In [19]:
columnasNaN = ['P3_2', 'P3_4', 'P3_7']

seccionIII = ModificarColumnasValor(df=seccionIII, cols=columnasNaN, valorR=np.nan, valorN=2)

columnasNaN = ['P3_3', 'P3_5', 'P3_6']

seccionIII = ModificarColumnasValor(df=seccionIII, cols=columnasNaN, valorR=np.nan, valorN=9)

### Borrar columnas

In [20]:
labels = ['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'COD_RES', 'DOMINIO', 'UPM', 'VIV_SEL', 'PROG',
          'N_REN', 'FAC_VIV', 'FAC_MUJ', 'ESTRATO', 'EST_DIS', 'UPM_DIS', 'P3_8']

seccionIII = BorrarColumnas(df=seccionIII, cols=labels)

### Pasar columnas categóricas a numéricas

In [21]:
lb = LabelEncoder()

In [22]:
seccionIII[['T_INSTRUM']] = seccionIII[['T_INSTRUM']].apply(transformColumn)
lb.classes_ 

array(['A1\r', 'A2\r', 'B1\r', 'B2\r', 'C1\r', 'C2\r'], dtype=object)

### Guardar dataset

In [23]:
GuardarDataset(df=seccionIII, name='datasets/sec_iii.csv')

## Seccion IX

### Cargar datos

In [24]:
ruta_parcial = "../../conjunto_de_datos_endireh_2016_csv/conjunto_de_datos_tb_sec_ix_endireh_2016/conjunto_de_datos/conjunto_de_datos_tb_sec_ix_endireh_2016.csv"

seccionIX = CargarPandasDataset(ruta_parcial)

seccionIX.iloc[random.choices(range(seccionIX.shape[0]), k=5)]

,ID_VIV,ID_MUJ,UPM,REN_M_ELE,VIV_SEL,PROG,HOGAR,DOMINIO,CVE_ENT,NOM_ENT,...,P9_8_10,P9_8_11,P9_8_12,P9_8_13,P9_8_14,FAC_VIV,FAC_MUJ,ESTRATO,UPM_DIS,EST_DIS
58955,3200710.04,3200710.04.01.02\r,3200710,2,4,96,1,U\r,32,Zacatecas\r,...,NaN,NaN,NaN,NaN,NaN,97,97,4,17743,6110
95837,2503587.02,2503587.02.01.02\r,2503587,2,2,74,1,U\r,25,Sinaloa\r,...,NaN,NaN,NaN,NaN,NaN,201,201,3,14166,4480
33038,1002621.08,1002621.08.01.02\r,1002621,2,8,193,1,C\r,10,Durango\r,...,NaN,NaN,NaN,NaN,NaN,119,237,2,6134,1700
21333,760629.07,0760629.07.01.02\r,760629,2,7,96,1,R\r,7,Chiapas\r,...,NaN,NaN,NaN,NaN,NaN,375,375,1,4018,1290
32482,1060118.20,1060118.20.01.03\r,1060118,3,20,30,1,R\r,10,Durango\r,...,2.0,2.0,NaN,NaN,NaN,155,309,1,6184,1730


### 2. Preservar los datos de las embarazadas

Borrar los registros de todas aquellas que repondieron que no estuvieron embarazadas entre Octubre 2011 hasta finales 2016 o que no se tenga resgistro de su respuesta. La manera fácil es solo preservar los registros de las que respondieron que sí estuvieron embaraadas en ese periodo de tiempo.

In [25]:
seccionIX = seccionIX[seccionIX.P9_2 == 1]

In [26]:
seccionIX.reset_index(drop=True, inplace=True) #reajustar el índice

### 3. Agregar columnas

En la pregunta 1 y 5, si todos los registros tienen valor 9 significa que no hay registro de la respuesta a esa pregunta, por ello se grega una columna que refleje esa información y todos los demás valores se hacen 0, así se preserva el _One Hot Encoding_.

#### Pregunta 1

In [27]:
seccionIX.insert(24, "P9_1_10", 0.0) #agregar columna con ceros

In [28]:
seccionIX.loc[seccionIX['P9_1_1']==9, 'P9_1_10'] = 1.0 #asignar los valores de columnas con 9

In [29]:
seccionIX.loc[seccionIX['P9_1_1']==9, [F'P9_1_{i}' for i in range(1,10)]] = 0.0 #los valores con 9 se hacen 0

#### Pregunta 5

In [30]:
seccionIX.insert(41, "P9_5_12", 0.0) #agregar columna con ceros

In [31]:
seccionIX.loc[seccionIX['P9_5_1']==9, 'P9_5_12']=1.0 #asignar los valores de columnas con 9

In [32]:
seccionIX.loc[seccionIX['P9_5_1']==9, [F'P9_5_{i}' for i in range(1,12)]]=0.0 #los valores con 9 se hacen 0

### Política de datos faltantes

In [33]:
columnasNaN = ['P9_8_11', 'P9_8_13', 'P9_8_14']

seccionIX = ModificarColumnasValor(df=seccionIX, cols=columnasNaN, valorR=np.nan, valorN=2)

seccionIX = ModificarColumnasValor(df=seccionIX, cols=['P9_7'], valorR=np.nan, valorN=99)

### 3.2 Crear clasificación

A partir de las respuestas de la pregunta 8, crear columna que va a servir de clasificación para los niveles de violencia obstétrica.

In [34]:
seccionIX_cls = InsertarColumnaNueva(df=seccionIX, nombreCol='P9_8', numeroCol=58, funcion=Clasificacion)

### Balancear clasificación

Que estén en mismo procentaje cada clasificación. Ya que el 73% de las mujeres no sufrieron violencia y las otras dos clasificaciones tiene 13% y 14%, obtendré una porción similar (15% porciento del total) para las no embarazadas. Para ello, debo obtener el 20% de las no violentadas.

Obtener el 20% de las que no sufrieron violencia

In [35]:
df_percent = seccionIX_cls.loc[seccionIX_cls['P9_8']==0].sample(frac=0.2)

Concatenar con las clasificaciones restantes.

In [36]:
seccionIX_cls = pd.concat([df_percent, seccionIX_cls.loc[seccionIX_cls['P9_8']!=0]])

In [37]:
seccionIX_cls.sort_index()
seccionIX_cls.reset_index(drop=True, inplace=True) #reajustar el índice

### Borrar columnas

In [38]:
labels = ['NOM_ENT', 'NOM_MUN', 'COD_RES_MU', 'COD_RES', 'DOMINIO', 'P9_2', 'T_INSTRUM']
labels.extend( [f'P9_8_{i}' for i in range(1,11)] )
labels.append('P9_8_12')

seccionIX_cls = BorrarColumnas(df=seccionIX_cls, cols=labels)

### Hacer columnas One Hot Encoding

In [39]:
columnasOHE = ['P9_8_11','P9_8_13'] 

seccionIX_cls = ModificarColumnasValor(df=seccionIX_cls, cols=columnasOHE, valorR=2, valorN=0)

### Guardar dataset

In [40]:
GuardarDataset(df=seccionIX_cls, name='datasets/sec_ix.csv')

## Unir datasets

In [41]:
result = pd.merge(seccionI, seccionII_reg, how="inner")
result = pd.merge(result, seccionIII, how="inner")
result = pd.merge(result, seccionIX_cls, how="inner")

### Borrar columnas

ID_MUJ no es necesario ya que no necesitamos identificar a las mujeres

In [42]:
result = BorrarColumnas(df=result, cols=['ID_MUJ'])

## Acomodar columnas

In [43]:
# las columnas en el orden actual
columnas = list(result.columns)

# columnas que pasare al inicio del dataframe
columnas_al_principio = ['ID_VIV', 'VIV_SEL',
                         'CVE_ENT', 'CVE_MUN', # si no se borran estas variables
                         'REGION', 'PAREN', 'SEXO', 'EDAD', 'NIV', 'GRA', 'DOMINIO', 
                         'HOGAR', 'T_INSTRUM', 'UPM', 'REN_M_ELE', 'PROG', 'FAC_VIV', 'FAC_MUJ', 
                         'ESTRATO', 'UPM_DIS', 'EST_DIS']

# quitar las columnas del lugar actual
for i in columnas_al_principio:
    columnas.remove(i)

# pasar al inicio de las columnas
columnas = columnas_al_principio+columnas
    
result = result[columnas]

In [44]:
result.columns

Index(['ID_VIV', 'VIV_SEL', 'CVE_ENT', 'CVE_MUN', 'REGION', 'PAREN', 'SEXO',
       'EDAD', 'NIV', 'GRA', 'DOMINIO', 'HOGAR', 'T_INSTRUM', 'UPM',
       'REN_M_ELE', 'PROG', 'FAC_VIV', 'FAC_MUJ', 'ESTRATO', 'UPM_DIS',
       'EST_DIS', 'P1_1', 'P1_2', 'P1_2_A', 'P1_3', 'P1_4_1', 'P1_4_2',
       'P1_4_3', 'P1_4_4', 'P1_4_5', 'P1_4_6', 'P1_4_7', 'P1_4_8', 'P1_4_9',
       'P1_5', 'P1_6', 'P1_7', 'P1_8', 'P1_9', 'P1_10_1', 'P1_10_2', 'P1_10_3',
       'P1_10_4', 'P2_5', 'P2_6', 'P2_8', 'P2_9', 'P2_10', 'P2_11', 'P2_12',
       'P2_13', 'P2_14', 'P2_15', 'P3_1', 'P3_2', 'P3_3', 'P3_4', 'P3_5',
       'P3_6', 'P3_7', 'P9_1_1', 'P9_1_2', 'P9_1_3', 'P9_1_4', 'P9_1_5',
       'P9_1_6', 'P9_1_7', 'P9_1_8', 'P9_1_9', 'P9_1_10', 'P9_3', 'P9_4_1',
       'P9_4_2', 'P9_4_3', 'P9_5_1', 'P9_5_2', 'P9_5_3', 'P9_5_4', 'P9_5_5',
       'P9_5_6', 'P9_5_7', 'P9_5_8', 'P9_5_9', 'P9_5_10', 'P9_5_11', 'P9_5_12',
       'P9_6', 'P9_7', 'P9_8_11', 'P9_8_13', 'P9_8_14', 'P9_8'],
      dtype='object')

### Guardar dataset

In [45]:
GuardarDataset(df=result, name='datasets/endireh.csv')